# This part is relevant for qestion 2.1


In [ ]:
!pip install mmh3
!pip install bitarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 6.1 MB/s eta 0:00:00


In [ ]:
# Python 3 program to build Bloom Filter
# Install mmh3 and bitarray 3rd party module first
# pip install mmh3
# pip install bitarray
import math
import mmh3
from bitarray import bitarray


class BloomFilter(object):

	'''
	Class for Bloom filter, using murmur3 hash function
	'''

	def __init__(self, items_count, fp_prob):
		'''
		items_count : int
			Number of items expected to be stored in bloom filter
		fp_prob : float
			False Positive probability in decimal
		'''
		# False possible probability in decimal
		self.fp_prob = fp_prob

		# Size of bit array to use
		self.size = self.get_size(items_count, fp_prob)

		# number of hash functions to use
		self.hash_count = self.get_hash_count(self.size, items_count)

		# Bit array of given size
		self.bit_array = bitarray(self.size)

		# initialize all bits as 0
		self.bit_array.setall(0)

	def add(self, item):
		'''
		Add an item in the filter
		'''
		digests = []
		for i in range(self.hash_count):

			# create digest for given item.
			# i work as seed to mmh3.hash() function
			# With different seed, digest created is different
			digest = mmh3.hash(item, i) % self.size
			digests.append(digest)

			# set the bit True in bit_array
			self.bit_array[digest] = True

	def check(self, item):
		'''
		Check for existence of an item in filter
		'''
		for i in range(self.hash_count):
			digest = mmh3.hash(item, i) % self.size
			if self.bit_array[digest] == False:

				# if any of bit is False then,its not present
				# in filter
				# else there is probability that it exist
				return False
		return True

	@classmethod
	def get_size(self, n, p):
		'''
		Return the size of bit array(m) to used using
		following formula
		m = -(n * lg(p)) / (lg(2)^2)
		n : int
			number of items expected to be stored in filter
		p : float
			False Positive probability in decimal
		'''
		m = -(n * math.log(p))/(math.log(2)**2)
		return int(m)

	@classmethod
	def get_hash_count(self, m, n):
		'''
		Return the hash function(k) to be used using
		following formula
		k = (m/n) * lg(2)

		m : int
			size of bit array
		n : int
			number of items expected to be stored in filter
		'''
		k = (m/n) * math.log(2)
		return int(k)


In [ ]:
def uniqe_items(random_list):
    ans = len(set(random_list))
    estimated_ans = 0

    #####TODO######
    #####Question 2.1############
    '''
    estimate the number of unique items by counting the number of zeros in a bloom filter.
    You are allowed to change the implementation of the bloom filter provided above.
    '''
    zerosCnt=0
    items_count = 100
    fp_prob = 0.01
    bloomFilter = BloomFilter(items_count, fp_prob)

    for item in random_list:
      bloomFilter.add(item)

    for value in bloomFilter.bit_array:
      if value == 0:
        zerosCnt += 1

    lenBitarray = bloomFilter.size
    if zerosCnt > 0:
      estimated_ans = -lenBitarray * math.log(zerosCnt / lenBitarray) / bloomFilter.hash_count
    return ans, estimated_ans

In [ ]:
items_count = 100
fp_prob = 0.01
bloom = BloomFilter(items_count, fp_prob)

import random

# Generate a list of 100 random numbers between 10 and 90
random_list = [str(random.randint(10, 90)) for _ in range(100)]
print(random_list)

for item in random_list:
    bloom.add(item)

print(bloom.check(random_list[0]))
print(bloom.check("0"))


['22', '35', '23', '20', '45', '65', '21', '19', '75', '84', '14', '77', '69', '43', '81', '76', '57', '88', '12', '28', '14', '19', '71', '54', '53', '33', '28', '65', '63', '60', '84', '70', '27', '60', '32', '75', '90', '55', '52', '30', '79', '47', '84', '71', '12', '20', '35', '40', '25', '13', '85', '68', '28', '35', '76', '17', '14', '17', '89', '35', '27', '50', '23', '59', '81', '60', '22', '88', '43', '81', '45', '66', '61', '70', '47', '68', '87', '32', '11', '58', '53', '22', '16', '43', '77', '25', '53', '43', '45', '80', '53', '58', '61', '37', '41', '59', '51', '60', '72', '26']
True
False


In [ ]:
uniqe_items(random_list)

(56, 56.14162084034904)

In [ ]:
# ----------- code for part 2.2 --------------
max_error = float('-inf')
min_error = float('inf')
total_error = 0
total_samples = 50

for i in range(total_samples):
    random_list = [str(random.randint(1, 50)) for _ in range(100)]
    true_uniques, estimated_uniques = uniqe_items(random_list)  # Replace with actual implementation
    # Compute absolute error
    error = abs(true_uniques - estimated_uniques)

    # Update max and min errors
    max_error = max(max_error, error)
    min_error = min(min_error, error)

    # Add to total error for average computation
    total_error += error

# Compute average error
avg_error = total_error / total_samples

# Print results
print(f"Maximum Error: {max_error}")
print(f"Minimum Error: {min_error}")
print(f"Average Error: {avg_error}")

Maximum Error: 1.277712676859089
Minimum Error: 0.03248129775275288
Average Error: 0.4829311102224247


In [ ]:
# ----------- code for part 2.3 --------------
import random
import mmh3
import math
from statistics import harmonic_mean

# FM Sketch Implementation
class FMSketch:
    def __init__(self, num_hash_func, bitmap_size, hash_functions): #todo this wasnt bitmap_size it was range, and I dont see we use them
        self.num_hash_func = num_hash_func
        self.bitmap_size = bitmap_size
        self.hash_functions = hash_functions
        self.counters = [0] * num_hash_func

    def add(self, item):
        print(f"=====Adding the item: {item}=====")
        for i in range(self.num_hash_func):
            hash_value = self.hash_functions[i](item)
            print(f"Hash value #{i} for item {item}: {hash_value}")
            trailing_zeros = self.count_trailing_zeros(hash_value)
            self.counters[i] = max(self.counters[i], trailing_zeros)

    def estimate(self):
        estimates = [2 ** trailing_zeros for trailing_zeros in self.counters]
        return sum(estimates)/len(estimates)

    def count_trailing_zeros(self, num):
        # Count trailing zeros in binary representation of num
        if num == 0:
            return 0
        binary_representation = bin(num)[2:]  # Convert to binary
        trailing_zeros = len(binary_representation) - len(binary_representation.rstrip('0'))
        return trailing_zeros


def h1(x):
    return (3 * x + 2) % 16

def h2(x):
    return (5 * x + 3) % 16

def h3(x):
    return (7 * x + 1) % 16

def h4(x):
    return (11 * x + 4) % 16


data_stream = [5, 12, 7, 5, 10, 7, 12, 7, 10, 12]
bitmap_size = 16
hash_functions = [h1, h2, h3, h4]

# TODO: Apply FM Sketch
fm_sketch = FMSketch(num_hash_func=4, bitmap_size=bitmap_size, hash_functions=hash_functions)

for item in data_stream:
    fm_sketch.add(item)

# TODO: Estimate the number of distinct elements
fm_estimate = fm_sketch.estimate()
# TODO: True count of unique items
true_count = len(set(data_stream))

print(f"\nTrue count of unique items: {true_count}")
print(f"FM Sketch estimate: {fm_estimate}")

=====Adding the item: 5=====
Hash value #0 for item 5: 1
Hash value #1 for item 5: 12
Hash value #2 for item 5: 4
Hash value #3 for item 5: 11
=====Adding the item: 12=====
Hash value #0 for item 12: 6
Hash value #1 for item 12: 15
Hash value #2 for item 12: 5
Hash value #3 for item 12: 8
=====Adding the item: 7=====
Hash value #0 for item 7: 7
Hash value #1 for item 7: 6
Hash value #2 for item 7: 2
Hash value #3 for item 7: 1
=====Adding the item: 5=====
Hash value #0 for item 5: 1
Hash value #1 for item 5: 12
Hash value #2 for item 5: 4
Hash value #3 for item 5: 11
=====Adding the item: 10=====
Hash value #0 for item 10: 0
Hash value #1 for item 10: 5
Hash value #2 for item 10: 7
Hash value #3 for item 10: 2
=====Adding the item: 7=====
Hash value #0 for item 7: 7
Hash value #1 for item 7: 6
Hash value #2 for item 7: 2
Hash value #3 for item 7: 1
=====Adding the item: 12=====
Hash value #0 for item 12: 6
Hash value #1 for item 12: 15
Hash value #2 for item 12: 5
Hash value #3 for ite

# This part is relevant for question 3.1 + 3.4


In [ ]:
import random


def minhash_signature(S, k):
    """
    Compute the MinHash signature for a given set S using k hash functions.
    Args:
        S : The input set of integers.
        k : The number of hash functions to use.
    Returns:
        The MinHash signature of set S.
    """
    signature = [-1]*k
    for i in range(k):
      h_i = hash_functions[i]
      for item in range(1,14):
        if h_i(item) in S:
          signature[i]= h_i(item)
          break

        # print(f"item {item} ==> {h_i(item)}")
    return signature

def jaccard_similarity(set1, set2):
    """
    Calculate the Jaccard similarity between two sets.
    Args:
        set1 : The first set.
        set2 : The second set.
    Returns:
        Jaccard similarity index.
    """
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    if union == 0:
      return 0.0
    return intersection / union


def minhash_algorithm(S, T, k):
    """
    Approximating the Jaccard Similarity for a given set S using k hash functions, using MinHash algorithm
    Args:
        S : The first input set of integers.
        T : The second input set of integers.
        k : The number of hash functions to use.
    Returns:
        The approximation of the Jaccard Similarity
    """
    count = 0
    for i in range(k):
      h_i = hash_functions[i]
      for item in range(1,14):
        if h_i(item) in S and h_i(item) in T:
          count += 1
          break
        elif h_i(item) in S or h_i(item) in T:
          break

        # print(f"item {item} ==> {h_i(item)}")
    return count/k


hash_functions = [
        lambda x: (2 * x + 1) % 11,  # h1(x)
        lambda x: (3 * x + 4) % 11,  # h2(x)
        lambda x: (5 * x + 2) % 11,  # h3(x)
        lambda x: (7 * x + 3) % 11,  # h4(x)
        lambda x: (9 * x + 6) % 11,  # h5(x)
    ]

A = {2, 5, 8, 9, 11}
B = {2, 4, 8, 9, 13}

k = 3

# TODO: calculate and print all the result from part 3.4 a-c

print(f"MinHash Signature for A: {minhash_signature(A, k)}")
print(f"MinHash Signature for B: {minhash_signature(B, k)}")

print(f"Jaccard Similarity for A and B: {jaccard_similarity(A, B)}")

print(f"Jaccard Similarity approximation unsing Minhash, k={k}: {minhash_algorithm(A, B, k)}")

k=5
print(f"Jaccard Similarity approximation unsing Minhash, k={k}: {minhash_algorithm(A, B, k)}")



MinHash Signature for A: [5, 2, 5]
MinHash Signature for B: [9, 2, 4]
Jaccard Similarity for A and B: 0.42857142857142855
Jaccard Similarity approximation unsing Minhash, k=3: 0.3333333333333333
Jaccard Similarity approximation unsing Minhash, k=5: 0.4
